# Sensitivity Analysis

We are going to use only three variables $I,\, P$ and $E$, each of them is a valid boolean string such that together they form a valid scenario (taking into account no more than 2 ones in each variable).

In [115]:
import numpy as np
import itertools
import pandas as pd
import warnings
import random
import datetime as dt
import time
import os
from os import listdir
from os.path import isfile,join
import matplotlib.pyplot as plt
from scipy.stats import t as tstats

## Creating input space
- $I$ and $P$ are boolean arrays of size three so that they have no 2 consecutive ones.
- $E$ is a boolean array of size 4. 

In total, there are $7*7*11=539$ possible scenarios.

In [116]:
num_options_per_category = {
    'investment' : 3,
    'policy' : 3,
    'event' : 4
}

In [117]:
def valid_scenarios_for_category(num_options):
    total_scenarios = list(itertools.product(range(2), repeat=num_options))
    valid_scenarios = [i for i in total_scenarios if sum(i)<3]
    return valid_scenarios

In [118]:
def generate_input_space_bool(category_options_dict):
    valid_scenarios_per_category_list = []
    for category, num_options in category_options_dict.items():
        valid_scenarios = valid_scenarios_for_category(num_options)
        valid_scenarios_per_category_list.append(valid_scenarios)
    
    # 539 input scenarios in dataset
    input_scenarios = list(itertools.product(*valid_scenarios_per_category_list))
    return input_scenarios

In [119]:
input_space = generate_input_space_bool(num_options_per_category)
input_space
len(input_space)

539

### Input space maps (bool <-> decimal)

In [120]:
def input_space_bool_to_decimal_dicts(category_options_dict):
    input_space_decimal_dict = {}
    
    for category, num_options in category_options_dict.items():
        valid_scenarios = valid_scenarios_for_category(num_options)
        num_scenarios = len(valid_scenarios)
        scenerios_decimal = list(range(0,num_scenarios,1))
        scenerios_decimal = [i/(num_scenarios - 1) for i in scenerios_decimal]
        bool_to_decimal_dict = dict(zip(valid_scenarios,scenerios_decimal))
        input_space_decimal_dict[category] = bool_to_decimal_dict
    
    return input_space_decimal_dict

In [121]:
def input_space_decimal_to_bool_dicts(category_options_dict):
    input_space_decimal_dict = {}
    
    for category, num_options in category_options_dict.items():
        valid_scenarios = valid_scenarios_for_category(num_options)
        num_scenarios = len(valid_scenarios)
        scenerios_decimal = list(range(0,num_scenarios,1))
        scenerios_decimal = [i/(num_scenarios - 1) for i in scenerios_decimal]
        bool_to_decimal_dict = dict(zip(scenerios_decimal,valid_scenarios))
        input_space_decimal_dict[category] = bool_to_decimal_dict
    
    return input_space_decimal_dict

In [122]:
bool_to_dec_dicts = input_space_bool_to_decimal_dicts(num_options_per_category)
bool_to_dec_dicts
bool_to_dec_dicts

{'investment': {(0, 0, 0): 0.0,
  (0, 0, 1): 0.16666666666666666,
  (0, 1, 0): 0.3333333333333333,
  (0, 1, 1): 0.5,
  (1, 0, 0): 0.6666666666666666,
  (1, 0, 1): 0.8333333333333334,
  (1, 1, 0): 1.0},
 'policy': {(0, 0, 0): 0.0,
  (0, 0, 1): 0.16666666666666666,
  (0, 1, 0): 0.3333333333333333,
  (0, 1, 1): 0.5,
  (1, 0, 0): 0.6666666666666666,
  (1, 0, 1): 0.8333333333333334,
  (1, 1, 0): 1.0},
 'event': {(0, 0, 0, 0): 0.0,
  (0, 0, 0, 1): 0.1,
  (0, 0, 1, 0): 0.2,
  (0, 0, 1, 1): 0.3,
  (0, 1, 0, 0): 0.4,
  (0, 1, 0, 1): 0.5,
  (0, 1, 1, 0): 0.6,
  (1, 0, 0, 0): 0.7,
  (1, 0, 0, 1): 0.8,
  (1, 0, 1, 0): 0.9,
  (1, 1, 0, 0): 1.0}}

In [123]:
dec_to_bool_dicts = input_space_decimal_to_bool_dicts(num_options_per_category)
dec_to_bool_dicts

{'investment': {0.0: (0, 0, 0),
  0.16666666666666666: (0, 0, 1),
  0.3333333333333333: (0, 1, 0),
  0.5: (0, 1, 1),
  0.6666666666666666: (1, 0, 0),
  0.8333333333333334: (1, 0, 1),
  1.0: (1, 1, 0)},
 'policy': {0.0: (0, 0, 0),
  0.16666666666666666: (0, 0, 1),
  0.3333333333333333: (0, 1, 0),
  0.5: (0, 1, 1),
  0.6666666666666666: (1, 0, 0),
  0.8333333333333334: (1, 0, 1),
  1.0: (1, 1, 0)},
 'event': {0.0: (0, 0, 0, 0),
  0.1: (0, 0, 0, 1),
  0.2: (0, 0, 1, 0),
  0.3: (0, 0, 1, 1),
  0.4: (0, 1, 0, 0),
  0.5: (0, 1, 0, 1),
  0.6: (0, 1, 1, 0),
  0.7: (1, 0, 0, 0),
  0.8: (1, 0, 0, 1),
  0.9: (1, 0, 1, 0),
  1.0: (1, 1, 0, 0)}}

In [124]:
def from_dec_to_bool(row):
    d_I = dec_to_bool_dicts['investment']
    d_P = dec_to_bool_dicts['policy']
    d_E = dec_to_bool_dicts['event']
    rbool = [d_I[row[0]], d_P[row[1]], d_E[row[2]]]
    return rbool

In [125]:
dec_to_bool_dicts['investment']

{0.0: (0, 0, 0),
 0.16666666666666666: (0, 0, 1),
 0.3333333333333333: (0, 1, 0),
 0.5: (0, 1, 1),
 0.6666666666666666: (1, 0, 0),
 0.8333333333333334: (1, 0, 1),
 1.0: (1, 1, 0)}

In [126]:
# input is a list of tuples [(1,0,0),(0,1,0),(1,0,0,0)]
# output 'CH1SP0SE0WE0BP1RE0CO1DI0WO0CS0'
def get_scenario_string(boolean_scenario): 
    bool_sc = list(sum(boolean_scenario,())) # flatten list
    list_events = ['CH','SP','SE','WE','BP','RE','CO','DI','WO','CS']
    f_n = [l+str(s) for l,s in zip(list_events,bool_sc)]
    f_n = ''.join(f_n)
    file_name = f_n
    return file_name

## Sampling
### Variance decomposition
We see this model as a function $f(X)=Y$, where the inputs are the scenarios, so $X\in\mathbb{R}^d$ and the output is a real value, that in our case, can be the emissions or the mobility choices. First, we are going to do an analysis on (overall) emissions and when we implement it, we can use the othe outputs.

The idea consists in estimating certain variances that we are going to define later in order to compute some sesitivity indices (there are first and second order sensitivity indices). We write
$$Var(Y)=\sum_{i=1}^dV_i+\sum_{i<j}^dV_{ij}+\ldots+V_{1,2, \ldots,d}$$where
$$V_i=Var_{x_i}(E_{X_{\sim i}}(Y|X))$$where the $X_{\sim i}$ notation means the set of all variables except $x_i$.

### First Order Index
$$S_i= \frac{V_i}{Var(Y)}$$We can interpret it as follows: "the fractional reduction in the variance of $Y$ which would be obtained on average if $X$ could be fixed".


In [127]:
def sampling(in_space,N):
    AB = random.sample(in_space,2*N)
    A = AB[0:N]
    B = AB[N:2*N]
    return A,B

In [128]:
def mat_AB_i(A,B): 
    d = 3
    N = len(A)
    An,Bn = np.array(A), np.array(B)
    AB_mat = np.zeros((d,N,d))
    for i in range(0,d):
        AB_mat[i] = An
        AB_mat[i][:,i] = Bn[:,i]
    return AB_mat

In [129]:
def generate_inputs_d(inputs, input_space_bool_to_decimal_dicts):
    inputs_d = []
    for i in inputs:
        dict_I = input_space_bool_to_decimal_dicts['investment']
        dict_P = input_space_bool_to_decimal_dicts['policy']
        dict_E = input_space_bool_to_decimal_dicts['event']
        t = [dict_I[i[0]], dict_P[i[1]], dict_E[i[2]]]
        inputs_d.append(t)
    return inputs_d

## Estimating
To compute the variance, we can use the following estimator: $$Var_{x_i}(E_{X_{\sim i}}(Y|X))\approx \frac{1}{N}\sum_{i=1}^Nf(B)_j\left(f(A_B^i)_j-f(A)_j\right)$$

First, get outputs and then compute the variance

In [130]:
# input is a list of tuples [(1,0,0),(0,1,0),(1,0,0,0)]
# output 'CH1SP0SE0WE0BP1RE0CO1DI0WO0CS0'
def get_scenario_string(boolean_scenario): 
    bool_sc = list(sum(boolean_scenario,())) # flatten list
    list_events = ['CH','SP','SE','WE','BP','RE','CO','DI','WO','CS']
    f_n = [l+str(s) for l,s in zip(list_events,bool_sc)]
    f_n = ''.join(f_n)
    file_name = f_n
    return file_name

In [131]:
# df: dataframe (e.g. total emissions df)
# M: matrix, make sure it is a list
def get_output(df, M):
    out_vector = []
    n = len(M)
    for i in range(0,n):
        dec_row = M[i]
        row_bool = from_dec_to_bool(dec_row)
        string = get_scenario_string(row_bool)
        cond = (df['scenario'] == string)
        outp = df[cond].total_emissions.values[0]
        out_vector.append(outp)
    return out_vector

In [132]:
# this function takes as input the matrices A and B
# and the original matrix ABi. 
# the index i at the end is the variable name, it can be 0, 1 or 2
def get_var_xi(df, A,B,ABi,i):
    ABi = ABi[i].tolist() # add '.astype(int)' in the middle if there are any probs with type
    out_A = get_output(df, A)
    out_B = get_output(df, B)
    out_ABi = get_output(df, ABi)    
    N = len(A)
    var = 0
    for j in range(0,N):
        var = out_B[j]*(out_ABi[j]-out_A[j])+var
    var = var/N
    return var

## Compute sensitivity indices

In [133]:
total_emissions_df = pd.read_csv("total_emissions.csv")

In [134]:
num_samples = 100

In [135]:
def calculate_sensitivity_indices(df, col_name, num_samples):
    total_samples = 2*num_samples
    input_space = generate_input_space_bool(num_options_per_category)
    input_bool_dec_dicts = input_space_bool_to_decimal_dicts(num_options_per_category)
    inputs_d = generate_inputs_d(input_space, input_bool_dec_dicts)
    A, B = sampling(inputs_d, total_samples)
#     AB = random.sample(inputs_d,total_samples)
#     A = AB[0:num_samples]
#     B = AB[num_samples:total_samples]
#     AB = np.zeros((3,num_samples,3))
#     Ant,Bnt = np.array(A), np.array(B)
#     AB[0] = Ant
#     AB[0][:,0] = Bnt[:,0]
    ABi = mat_AB_i(A,B)
    
    varI = get_var_xi(df, A,B,ABi,0)
    varP = get_var_xi(df, A,B,ABi,1)
    varE = get_var_xi(df, A,B,ABi,2)
    
    varY = df[col_name].var(axis=0)
    
    S_I = varI / varY
    S_P = varP / varY
    S_E = varE / varY
    
    sensitivity_indices_dict = {
        'investment' : S_I,
        'policy' : S_P,
        'event' : S_E
    }
    
    return sensitivity_indices_dict

In [136]:
sensitivity_indices = calculate_sensitivity_indices(total_emissions_df, 'total_emissions', num_samples)
sensitivity_indices

{'investment': -0.20471642413622088,
 'policy': 0.08586955444575066,
 'event': 0.39496765449098337}

In [137]:
for category_str, sensitivity_index in sensitivity_indices.items():
    print(f'- The sensitivity of the {category_str} category is {str(sensitivity_index)}')

- The sensitivity of the investment category is -0.20471642413622088
- The sensitivity of the policy category is 0.08586955444575066
- The sensitivity of the event category is 0.39496765449098337


### Calculate confidence intervals

In [138]:
# dataset input needs to be np array
def calculate_confidence_intervals(dataset, confidence):
    mean = dataset.mean() 
    s_dev = dataset.std() 
    num_datapoints = len(dataset)
    dof = num_datapoints-1 
    t_crit = np.abs(tstats.ppf((1-confidence)/2,dof))
    confidence_interval = (mean-s_dev*t_crit/np.sqrt(num_datapoints), mean+s_dev*t_crit/np.sqrt(num_datapoints)) 
    return confidence_interval

### Sensitivity Analysis 
Multiple runs of computing the sensitivity indices for each category with a given number of samples.
Compute the mean and the 95% confidence interval for each category

In [139]:
def sensitivity_analysis(num_runs, df, col_name, num_samples):
    S1_I_arr = []
    S1_P_arr = []
    S1_E_arr = []
    for i in range (0, num_runs):
        sensitivity_indices = calculate_sensitivity_indices(total_emissions_df, 'total_emissions', 8)
        S1_I_arr.append(sensitivity_indices['investment'])
        S1_P_arr.append(sensitivity_indices['policy'])
        S1_E_arr.append(sensitivity_indices['event'])
        
    S1_I_np_arr = np.array(S1_I_arr)
    S1_I = S1_I_np_arr.mean()
    S1_I_conf = calculate_confidence_intervals(S1_I_np_arr, 0.95)
    S1_P_np_arr = np.array(S1_P_arr)
    S1_P = S1_P_np_arr.mean()
    S1_P_conf = calculate_confidence_intervals(S1_P_np_arr, 0.95)
    S1_E_np_arr = np.array(S1_E_arr)
    S1_E = S1_E_np_arr.mean()
    S1_E_conf = calculate_confidence_intervals(S1_E_np_arr, 0.95)
    
    sensitivity_indices_dict = {
        'investment' : {
            'S1' : S1_I,
            'S1_conf' : S1_I_conf
        },
        'policy' : {
            'S1' : S1_P,
            'S1_conf' : S1_P_conf
        },
        'event' : {
            'S1' : S1_E,
            'S1_conf' : S1_E_conf    
        }
    }
    return sensitivity_indices_dict

In [140]:
num_runs = 2000

In [141]:
sa_result = sensitivity_analysis(num_runs, total_emissions_df, 'total_emissions', num_samples)

In [142]:
print(sa_result)

{'investment': {'S1': 0.06067976756086243, 'S1_conf': (0.013467257514839702, 0.10789227760688516)}, 'policy': {'S1': 1.2232318806860702, 'S1_conf': (0.736283541211185, 1.7101802201609555)}, 'event': {'S1': -0.038310546110087204, 'S1_conf': (-0.16295469778489208, 0.08633360556471768)}}


In [143]:
print(f'SENSITIVITY ANALYSIS FOR {str(num_runs)} RUNS, SAMPLE SIZE = {str(num_samples)}')
for category_str, stats in sa_result.items():
    print(f' {category_str}')
    for stat_key, value in stats.items():
        print(f' - {stat_key}: {value}')

SENSITIVITY ANALYSIS FOR 2000 RUNS, SAMPLE SIZE = 100
 investment
 - S1: 0.06067976756086243
 - S1_conf: (0.013467257514839702, 0.10789227760688516)
 policy
 - S1: 1.2232318806860702
 - S1_conf: (0.736283541211185, 1.7101802201609555)
 event
 - S1: -0.038310546110087204
 - S1_conf: (-0.16295469778489208, 0.08633360556471768)
